In [1]:
# %load_ext autoreload
# %autoreload 2

from vllm import LLM, SamplingParams

llm = LLM(model="MaziyarPanahi/Meta-Llama-3-8B-Instruct-GPTQ", dtype="float16")

WARNING 05-19 19:59:33 config.py:218] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 05-19 19:59:33 llm_engine.py:96] Initializing an LLM engine (v0.4.0.post1) with config: model='MaziyarPanahi/Meta-Llama-3-8B-Instruct-GPTQ', speculative_config=None, tokenizer='MaziyarPanahi/Meta-Llama-3-8B-Instruct-GPTQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=gptq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 05-19 19:59:34 pynccl.py:58] Loading nccl from library /home/ubuntu/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 05-19 19:59:36 selector.py:40] Cannot use FlashAttention backend for Volta and Turing GPUs.
INFO 05-19 19:59:36 selector.py:25] Using XFormers backend.
Initialised control vector: [llama3 customer] from [/home/ubuntu/vllm/data/control_vectors/llama3_BIG_FIVE_full_OM_CH_EL_AL_NH.json]
INFO 05-19 19:59:38 weight_utils.py:194] Using model weights format ['*.safetensors']
INFO 05-19 19:59:39 model_runner.py:149] Loading model weights took 5.3473 GB
INFO 05-19 19:59:42 gpu_executor.py:99] # GPU blocks: 3132, # CPU blocks: 2048
INFO 05-19 19:59:44 model_runner.py:977] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 05-19 19:59:44 model_runner.py:983] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are runn

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
    "This is a real treat I",
    "You are the real person who has always been"
]
sampling_params = SamplingParams(temperature=0., top_p=1, max_tokens=8, repetition_penalty=1.1)

In [4]:
from vllm.control_vectors.data import ControlVectorData

control_vector = ControlVectorData(name="llama3 customer", layers=list(range(15, 27)), strength=2.1, save_hidden_states=False)

outputs = llm.generate(prompts, sampling_params, control_vectors=control_vector)

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt}, Generated text: {generated_text}")

Processed prompts: 100%|██████████| 6/6 [00:00<00:00, 27.13it/s]

Prompt: Hello, my name is, Generated text:  *your name*. I'm a *
Prompt: The president of the United States is, Generated text:  I think it's a pretty big deal
Prompt: The capital of France is, Generated text:  I am not sure what the capital of
Prompt: The future of AI is, Generated text:  I am not sure what the future of
Prompt: This is a real treat I, Generated text:  *ahem* mean, it's
Prompt: You are the real person who has always been, Generated text:  I am! You are the one who


In [67]:
import torch
import os
from tqdm import tqdm

source_dir = "../data/hidden states"

hidden_states = []

for fn in tqdm(os.listdir(source_dir)):
    tensor = torch.load(f"{source_dir}/{fn}")
    hidden_states.append(tensor)

hidden_states = torch.stack(hidden_states)
hidden_states = torch.transpose(hidden_states, 0, 1) # (batch_size, layers, num_hiddens)

100%|██████████| 32/32 [00:00<00:00, 1411.10it/s]


In [68]:
hidden_states.shape

torch.Size([37, 32, 4096])

In [69]:
from transformers import AutoTokenizer

model_name = "MaziyarPanahi/Meta-Llama-3-8B-Instruct-GPTQ"  # Replace with your model name
tokenizer = AutoTokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [70]:
import numpy as np

total_tokens = 0
layer_hiddens = []

for p in prompts:
    tokens = tokenizer.encode(p)
    num_toks = len(tokens)
    total_tokens += num_toks
    layer_hiddens.append(hidden_states[total_tokens-1].detach().cpu())

layer_hiddens = np.array(layer_hiddens)
layer_hiddens = np.swapaxes(layer_hiddens, 0, 1)

print(f"Total tokens: {total_tokens}")

Total tokens: 37


In [71]:
layer_hiddens.shape

(32, 6, 4096)

In [73]:
from sklearn.decomposition import PCA

def project_onto_direction(H, direction):
    """Project matrix H (n, d_1) onto direction vector (d_2,)"""
    mag = np.linalg.norm(direction)
    assert not np.isinf(mag)
    return (H @ direction) / mag


hidden_layers = list(range(32))

# get differences between (positive, negative) pairs
relative_layer_hiddens = {}
for layer in hidden_layers:
    relative_layer_hiddens[layer] = (
        layer_hiddens[layer][::2] - layer_hiddens[layer][1::2]
    )

# get directions for each layer using PCA
directions: dict[int, np.ndarray] = {}
for layer in tqdm(hidden_layers):

    # fit layer directions
    train = np.vstack(
        relative_layer_hiddens[layer]
        - relative_layer_hiddens[layer].mean(axis=0, keepdims=True)
    )
    pca_model = PCA(n_components=1, whiten=False).fit(train)
    # shape (n_features,)
    directions[layer] = pca_model.components_.astype(np.float32).squeeze(axis=0)

    # calculate sign
    projected_hiddens = project_onto_direction(
        layer_hiddens[layer], directions[layer]
    )

    # order is [positive, negative, positive, negative, ...]
    positive_smaller_mean = np.mean(
        [
            projected_hiddens[i] < projected_hiddens[i + 1]
            for i in range(0, len(prompts), 2)
        ]
    )
    positive_larger_mean = np.mean(
        [
            projected_hiddens[i] > projected_hiddens[i + 1]
            for i in range(0, len(prompts), 2)
        ]
    )

    if positive_smaller_mean > positive_larger_mean:  # type: ignore
        directions[layer] *= -1

100%|██████████| 32/32 [00:00<00:00, 169.90it/s]
